# Overview

Similar to other examples that we had in the course - this notebook will demo how to scrape [Books to scrape](https://books.toscrape.com/index.html) - a fictional demo site that shows book prices and is designed for teaching web scraping! As it looks like a retailer website!

## Can we scrape at all?

Before staring out - we normally check whether we can scrape the site. To validate whether this site allows scraping - let's check this as well. This way we also demonstrate common steps:
* Firstly we can check [https://books.toscrape.com/robots.txt](https://books.toscrape.com/robots.txt) for the site - which does not exist! 
* Secondly we check the Terms and Conditions on the site - however this site does not have anything. Searching the internet tells us that [this is a popular scraping sandbox for beginners](https://proxyway.com/guides/best-websites-to-practice-your-web-scraping-skills)!

Thus we can proceed!

----------------------

# Explore the site

First off - lets import the necessary things into the notebook

In [31]:
import time
import random
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
import math

Then let's set up the user agent that will be set up with the scraper request. We can build off [what we saw in Session 10 for inspiration](https://xtophe.notion.site/Web-Scraping-for-CPI-Training-c36b27c2ea6b4f02ad760069afaf6fb5).

In [32]:
heads = {
    'User-Agent':'ESCAP Webscraping RAP demo scraper 1.0',
    'email': 'example@gmail.com',
    'Accept-Language': 'en-US, en;q=0.5'}

s = requests.Session()

## Navigation and how to get the data

Firstly, using the developer mode in the browser, we see that there doesn't seem to be an API behind this site as only the `html` is delivered. Thus we will need to use `BeautifulSoup` and scrape the full `html`.

To navigate the `html` aspect of the site, it looks like there are two ways we can do it:

### Single-shot approach and pagination
It looks like off the bat, the site lists all books together with a `next` page at the bottom right.

When we start on page 1, within the html we see a `pager` class and `next` class within it:

<img src="../docs/images/books_to_scrape_approach_page_1.png" alt="Alternative approach using navigation" style="box-shadow: 5px 5px 5px gray;" width="500">

<img src="../docs/images/books_to_scrape_approach_pagination_html.png" alt="Alternative approach using navigation" style="box-shadow: 5px 5px 5px gray;" width="500">
<!-- 
![](../docs/images/books_to_scrape_approach_page_1.png)
![](../docs/images/books_to_scrape_approach_pagination_html.png) -->

Once we get to the end, we see only a `previous` and the `next` class is no longer there

<img src="../docs/images/books_to_scrape_pagination_last_page.png" alt="Alternative approach using navigation" style="box-shadow: 5px 5px 5px gray;" width="500">

<img src="../docs/images/books_to_scrape_approach_pagination_html_last_page.png" alt="Alternative approach using navigation" style="box-shadow: 5px 5px 5px gray;" width="500">

<!-- ![](../docs/images/books_to_scrape_pagination_last_page.png)
![](../docs/images/books_to_scrape_approach_pagination_html_last_page.png) -->

The URL also starts behaves in a stable way, whether for all books or for a specific categories:
* https://books.toscrape.com/catalogue/category/books_1/page-2.html
* https://books.toscrape.com/catalogue/category/books/fantasy_19/page-2.html

This means that we can iterate through the pages quite easily. Since all the books are also available from the main page - we could just iterate through all 50 pages and get all 1000 books on the site.

### Category-by-category approach
We can also navigate the category section on the left and then iterate through all the pages for the category:

<img src="../docs/images/books_to_scrape_approach_2.png" alt="Alternative approach using navigation" style="box-shadow: 5px 5px 5px gray;" width="350">

<!-- ![](../docs/images/books_to_scrape_approach_2.png) -->

The html class for this is `side_categories` and we should easily be able to get into the unordered list and iterate through all of the categories:

<img src="../docs/images/books_to_scrape_approach_2_html.png" alt="HTML listing the categories" style="box-shadow: 5px 5px 5px gray;" width="500">
<!-- ![](../docs/images/books_to_scrape_approach_2_html.png) -->

### Which way should we go?
As many retailer websites around the world NSOs will need to scrape will likely not get a single home page that lists all products - the more realistic scenario is to iterate first through each category, and then through each page on that category.


## Scraping info on each individual book page

On each individual page, there are likely several categories that are of interest to us for consumer price statistics:
* Product name
* Product category
* Product description 
* UPC
* Final (post-tax) price
* Whether the product is available or not

<img src="../docs/images/books_to_scrape_individidual_page_info_of_interest.png" alt="HTML listing the categories" style="box-shadow: 5px 5px 5px gray;" width="800">

Below we will go through how to get all this information.

# Exploring how to scrape the site

## Scraping categories

In [33]:
# Specify the URL
shop_url = "https://books.toscrape.com/"

# Use the with clause we learned about (could also be done directly) to collect and parse the site
with s.get(shop_url, headers=heads) as res:
    response = BeautifulSoup(res.text, "html.parser")

In [36]:
type(response) == BeautifulSoup

True

Given the navigation we found previously (i.e. the `side_categories` div class), lets find all the categories:

In [9]:
# Focus just on the section we want
side_category_section = response.find("div", class_ = "side_categories")

# Isolate all the categories using the link tag as they will have a link
categories = side_category_section.find_all('a')

# Iterate through all the categories and save the link in a dictionary key-value pair assigned
# to the name of the category itself
dictionary_of_categories = {}
for category in categories:
    dictionary_of_categories[category.text.strip()] = category.get('href')

# Now lets see what we were able to scrape
dictionary_of_categories


{'Books': 'catalogue/category/books_1/index.html',
 'Travel': 'catalogue/category/books/travel_2/index.html',
 'Mystery': 'catalogue/category/books/mystery_3/index.html',
 'Historical Fiction': 'catalogue/category/books/historical-fiction_4/index.html',
 'Sequential Art': 'catalogue/category/books/sequential-art_5/index.html',
 'Classics': 'catalogue/category/books/classics_6/index.html',
 'Philosophy': 'catalogue/category/books/philosophy_7/index.html',
 'Romance': 'catalogue/category/books/romance_8/index.html',
 'Womens Fiction': 'catalogue/category/books/womens-fiction_9/index.html',
 'Fiction': 'catalogue/category/books/fiction_10/index.html',
 'Childrens': 'catalogue/category/books/childrens_11/index.html',
 'Religion': 'catalogue/category/books/religion_12/index.html',
 'Nonfiction': 'catalogue/category/books/nonfiction_13/index.html',
 'Music': 'catalogue/category/books/music_14/index.html',
 'Default': 'catalogue/category/books/default_15/index.html',
 'Science Fiction': 'cata

Fantastic! Appending each category to the site URL will give us a way how to navigate to the category!

We can also probably remove the `index.html` from the end of each as it is detrimental to the perfomance of the site.

## Navigate all pages and save the product (book) 

Lets say we want to focus on https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html. as there are 75 books to scrape within this category (others should look the same).

### Finding out the product URL so that we can go there later

In [10]:
# Scrape the main site URL + the category we want in the catalogue
category_url = "catalogue/category/books/mystery_3/index.html"
with s.get(shop_url + category_url, headers=heads) as res:
    response = BeautifulSoup(res.text, "html.parser")

In [11]:
products = response.find_all("article", class_="product_pod")
# Extract the first link (which happens to be the picture), although we can get the second link too
products[0].find('a').get('href')

# as the full site per book is 
# https://books.toscrape.com/catalogue/scott-pilgrims-......html
# we should thus strip ../../.. but keep the catalogue
shop_url+'catalogue'+products[0].find('a').get('href')[8:]

'https://books.toscrape.com/catalogue/sharp-objects_997/index.html'

Perfect! So we now know all the URLs per product

### Finding the number of pages to go through

Now we need to know how to iterate through pages

In [12]:
# there is no 'number of pages' to check, but we can estimate the number from 
# the amount of results at the top left of the page. For instance:
response.find("form", class_="form-horizontal").text.strip()

'32 results - showing 1 to 20.'

In [13]:
# splitting with spaces and then extracting the first item in the list gets us the number by category
number_of_products = response.find("form", class_="form-horizontal").text.strip().split(" ")[0]
#knowing that each page displays 20 products and rounding up gives us the number of pages
round(int(number_of_products)/20)


2

### Putting this all together to scrape the whole category

In [14]:
# Scrape the main site URL + the category we want in the catagory of interest
category_url = "catalogue/category/books/historical-fiction_4/"
with s.get(shop_url + category_url, headers=heads) as res:
    response = BeautifulSoup(res.text, "html.parser")

# create empty list to save URLs of each product/book to scrape into a list
products_to_scrape = []

# Find the number of pages to iterate through:
number_of_products = response.find("form", class_="form-horizontal").text.strip().split(" ")[0]
# iterate through the list of pages
for page in range(0,math.ceil(int(number_of_products)/20)):
    # as we start off scraping the category site anyway, then we don't need to scrape it again
    # however if we are now on page 2, we haven't yet scraped it so we should
    if page > 0:
        category_url = category_url + "page-{}.html".format(page+1)
        print('getting',shop_url+category_url)
        with s.get(shop_url + category_url, headers=heads) as res:
            response = BeautifulSoup(res.text, "html.parser")

    # Find all the product pods on this page
    products = response.find_all("article", class_="product_pod")
    # Extract the first link (which happens to be the picture), although we can get the second link too
    for product in products:
        href = product.find('a').get('href')
        products_to_scrape.append(shop_url+'catalogue'+product.find('a').get('href')[8:])


getting https://books.toscrape.com/catalogue/category/books/historical-fiction_4/page-2.html


In [15]:
# Check the first 3 products
products_to_scrape[3]

'https://books.toscrape.com/catalogue/the-house-by-the-lake_846/index.html'

In [16]:
# Double check the number sraped
len(products_to_scrape)

26

Perfect! As this page has 75 products/books - we scraped them all!

## Scaping the individual product page

In [17]:
# Scrape the main site URL + the category we want in the catagory of interest
category_url = "catalogue/mesaerion-the-best-science-fiction-stories-1800-1849_983/index.html"
with s.get(shop_url + category_url, headers=heads) as res:
    response = BeautifulSoup(res.text, "html.parser")

### Get product (i.e. book) title

In [18]:
# get product/book name
response.title.text.split("|")[0].strip()

'Mesaerion: The Best Science Fiction Stories 1800-1849'

### Get Product description

In [19]:
# description is challenging as it has no unique id or class, thus we could
# find the product description tab (which is the first sub-header class) and
# use the `find_next()` method to get to the description
print(response.find_all("div", class_="sub-header")[0].find_next('p').text)

Andrew Barger, award-winning author and engineer, has extensively researched forgotten journals and magazines of the early 19th century to locate groundbreaking science fiction short stories in the English language. In doing so, he found what is possibly the first science fiction story by a female (and it is not from Mary Shelley). Andrew located the first steampunk short Andrew Barger, award-winning author and engineer, has extensively researched forgotten journals and magazines of the early 19th century to locate groundbreaking science fiction short stories in the English language. In doing so, he found what is possibly the first science fiction story by a female (and it is not from Mary Shelley). Andrew located the first steampunk short story, which has not been republished since 1844. There is the first voyage to the moon in a balloon, republished for the first time since 1820 that further tells of a darkness machine and a lunarian named Zuloc. Other sci-stories include the first r

### Get product information

As this is a table, there are 2 ways of getting this information, via `BeautifulSoup` and via `pandas`:

**`BeautifulSoup` approach**

In [20]:
# save the table into a dictionary
product_info = {}

# filter to the product page
product_page = response.find("article", class_="product_page")
# as this is the only table with rows, we can just get find all rows on the product page
rows = product_page.find_all('tr')
for row in rows:
    # focus on each row's cells and then save the values
    cells = row.find_all(['th', 'td'])
    product_info[cells[0].text] = cells[1].text
    
# have a look at what we saved
product_info

{'UPC': 'e30f54cea9b38190',
 'Product Type': 'Books',
 'Price (excl. tax)': 'Â£37.59',
 'Price (incl. tax)': 'Â£37.59',
 'Tax': 'Â£0.00',
 'Availability': 'In stock (19 available)',
 'Number of reviews': '0'}

**`pandas` approach**

In [21]:
all_tables = pd.read_html(shop_url + category_url)
all_tables[0]

,0,1
0,UPC,e30f54cea9b38190
1,Product Type,Books
2,Price (excl. tax),£37.59
3,Price (incl. tax),£37.59
4,Tax,£0.00
5,Availability,In stock (19 available)
6,Number of reviews,0


In [22]:
# we can convert this to a dictionary by first setting the first 
# column as the index and second by telling pandas to convert it to a dictionary
all_tables[0].set_index(0).to_dict()[1]

{'UPC': 'e30f54cea9b38190',
 'Product Type': 'Books',
 'Price (excl. tax)': '£37.59',
 'Price (incl. tax)': '£37.59',
 'Tax': '£0.00',
 'Availability': 'In stock (19 available)',
 'Number of reviews': '0'}

# Putting it all together: scraping the whole site

Now that we've explored it all - we can put it together and create a scraper for the whole site!

However instead of trying to do everything in one long script, lets at least break it out into two jobs:
1. a script that checks the entire site and saves the URLs of each product that needs to be scraped, and
2. a script that uses the URL of the product by getting all product characteristics that need saving - and this we already did above!

Thus #1 is the main thing that we need to do - i.e. find all the product URLs

In [23]:
# Specify the URL
shop_url = "https://books.toscrape.com/"

# Use the with clause we learned about (could also be done directly) to 
# collect and parse the site
with s.get(shop_url, headers=heads) as res:
    response = BeautifulSoup(res.text, "html.parser")

# Get all categories
side_category_section = response.find("div", class_ = "side_categories")

# Isolate all the categories using the link tag as they will have a link
categories_found = side_category_section.find_all('a')

# Iterate through all the categories and inside each through each page
# to save the link of the product in a dictionary
dictionary_of_products = {}

for categories in categories_found:
    # Since the 'Books' category is a catch-all, we want to skip it
    if categories.text.strip() == 'Books':
        continue

    # For the category, create a dictionary to nest that will save the relevant info
    dictionary_of_products[categories.text.strip()] = {}
    
    # Save the URL of the category within this nested dictionary
    dictionary_of_products[categories.text.strip()]['category_url'] = categories.get('href')[:-10]

    # break
    # Initiate an empty list of urls to scrape for the category
    products_to_scrape_list = []
    for i, each_category in enumerate(categories):
        # get the category page
        with s.get(shop_url + categories.get('href')[:-10], headers=heads) as res:
            response = BeautifulSoup(res.text, "html.parser")

        # Find the number of pages to iterate through:
        number_of_products = response.find("form", class_="form-horizontal").text.strip().split(" ")[0]
        # iterate through the list of pages
        for page in range(0,math.ceil(int(number_of_products)/20)):
            # as we start off scraping the category site anyway, then we don't need to scrape it again
            # however if we are now on page 2, we haven't yet scraped it so we should
            if page > 0:
                category_url = categories.get('href')[:-10] + "page-{}.html".format(page+1)
                with s.get(shop_url + category_url, headers=heads) as res:
                    response = BeautifulSoup(res.text, "html.parser")

            # Find all the product pods on this page
            products = response.find_all("article", class_="product_pod")
            # Extract the first link (which happens to be the picture), although we can get the second link too
            for product in products:
                href = product.find('a').get('href')
                products_to_scrape_list.append(shop_url+'catalogue'+product.find('a').get('href')[8:])

        # Save the nominal number of products and the list of urls in the main dictionary
        dictionary_of_products[categories.text.strip()]['nominal_number_of_products'] = number_of_products
        dictionary_of_products[categories.text.strip()]['products_to_scrape_list'] = products_to_scrape_list


Great, so we've scraped the entire site - and saved the list of products into a list by category - in a dictionary called `dictionary_of_products`. Let's check if there were no mistakes and that we actually saved 1000 product URLs

In [25]:
# do a little counter - starting with zero
counter = 0
# go through each category
for category in dictionary_of_products:
    # and add to the counter the number of products
    counter += len(dictionary_of_products[category]['products_to_scrape_list'])

# now display the total
counter

1000

In [30]:
# lets see what it looks like for one category (as it would be really busy if we looked at a few)
dictionary_of_products['Travel']

{'category_url': 'catalogue/category/books/travel_2/',
 'nominal_number_of_products': '11',
 'products_to_scrape_list': ['https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html',
  'https://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html',
  'https://books.toscrape.com/catalogue/see-america-a-celebration-of-our-national-parks-treasured-sites_732/index.html',
  'https://books.toscrape.com/catalogue/vagabonding-an-uncommon-guide-to-the-art-of-long-term-world-travel_552/index.html',
  'https://books.toscrape.com/catalogue/under-the-tuscan-sun_504/index.html',
  'https://books.toscrape.com/catalogue/a-summer-in-europe_458/index.html',
  'https://books.toscrape.com/catalogue/the-great-railway-bazaar_446/index.html',
  'https://books.toscrape.com/catalogue/a-year-in-provence-provence-1_421/index.html',
  'https://books.toscrape.com/catalogue/the-road-to-little-dribbling-adventures-of-an-american-in-britain-notes-f

Perfect, the total is 1000, so we got all product URLs! 

# Conclusion 

This notebook has demonstrated how to explore and parse the information on the site and get what we need. While we did not make a complex script that did everything all in one script, there is actually a very good reason -- this script would be big, hard to code, hard to understand, and hard to debug as we're likely to have mistakes. This would also make it a very RAP-unfriendly way to code. Thus we will see how the whole scraper can be developed to be more robust.